In [1]:
import sys, os
import numpy as np
np.set_printoptions(threshold=np.inf)
import pandas as pd
from utils.utils import read_txt, read_txt_np_int
from functools import lru_cache
from math import ceil, floor

# INPUT

In [2]:
inputfilename = './inputs/day15A.txt'

inputdata = read_txt(inputfilename)

testdata_small = ["########",
"#..O.O.#",
"##@.O..#",
"#...O..#",
"#.#.O..#",
"#...O..#",
"#......#",
"########",
"",
"<^^>>>vv<v>>v<<"]

testdata_large = ["##########",
"#..O..O.O#",
"#......O.#",
"#.OO..O.O#",
"#..O@..O.#",
"#O#..O...#",
"#O..O..O.#",
"#.OO.O.OO#",
"#....O...#",
"##########",
"",
"<vv>^<v^>v>^vv^v>v<>v^v<v<^vv<<<^><<><>>v<vvv<>^v^>^<<<><<v<<<v^vv^v>^",
"vvv<<^>^v^^><<>>><>^<<><^vv^^<>vvv<>><^^v>^>vv<>v<<<<v<^v>^<^^>>>^<v<v",
"><>vv>v^v^<>><>>>><^^>vv>v<^^^>>v^v^<^^>v^^>v^<^v>v<>>v^v^<v>v^^<^^vv<",
"<<v<^>>^^^^>>>v^<>vvv^><v<<<>^^^vv^<vvv>^>v<^^^^v<>^>vvvv><>>v^<<^^^^^",
"^><^><>>><>^^<<^^v>>><^<v>^<vv>>v>>>^v><>^v><<<<v>>v<v<v>vvv>^<><<>^><",
"^>><>^v<><^vvv<^^<><v<<<<<><^v<<<><<<^^<v<^^^><^>>^<v^><<<^>>^v<v^v<v^",
">^>>^v>vv>^<<^v<>><<><<v<<v><>v<^vv<<<>^^v^>^^>>><<^v>>v^v><^^>>^<>vv^",
"<><^^>^^^<><vvvvv^v<v<<>^v<v>v<<^><<><<><<<^^<<<^<<>><<><^^^>^^<>^>v<>",
"^^>vv<^v^v<vv>^<><v<^v>^^^>>>^^vvv^>vvv<>>>^<^>>>>>^<<^v>^vvv<>^<><<v>",
"v^^>>><<^^<>>^v^<v^vv<>v^<<>^<^v^v><^<<<><<^<v><v<>vv>>v><v^<vv<>v^<<^"]

In [138]:
data_to_use = inputdata
warehouse_map = np.array([[x for x in line] for line in data_to_use if '#' in line])

print(warehouse_map)

robot_instructions_list = [x for line in data_to_use for x in line if '<' in line or '>' in line or 'v' in line or '^' in line]

robot_position = tuple(np.array(np.where(warehouse_map == "@")).flatten())

warehouse_map[robot_position] = '.'

print(warehouse_map, robot_position, robot_instructions_list)

[['#' '#' '#' '#' '#' '#' '#' '#' '#' '#' '#' '#' '#' '#' '#' '#' '#' '#'
  '#' '#' '#' '#' '#' '#' '#' '#' '#' '#' '#' '#' '#' '#' '#' '#' '#' '#'
  '#' '#' '#' '#' '#' '#' '#' '#' '#' '#' '#' '#' '#' '#']
 ['#' '.' '.' '.' '#' '#' '.' '.' '.' 'O' 'O' 'O' '.' '#' '.' 'O' '.' 'O'
  'O' 'O' '.' '.' '.' '.' '.' '.' 'O' '#' '#' '#' '.' '.' '.' 'O' '.' '#'
  '.' '.' 'O' '.' '.' '.' '.' '.' '.' '.' 'O' '.' '.' '#']
 ['#' '.' '.' '.' '.' 'O' '.' '.' '.' 'O' '.' '.' '.' '.' '.' 'O' '.' '.'
  'O' '#' 'O' '#' 'O' '.' 'O' '.' '.' '.' '.' 'O' '.' '.' '.' 'O' 'O' '#'
  'O' 'O' 'O' 'O' '.' '.' '.' '.' 'O' 'O' 'O' '.' '.' '#']
 ['#' '.' '.' 'O' '.' 'O' '.' '.' 'O' '.' '#' '.' '.' '#' '.' '.' '.' 'O'
  '.' '.' '.' 'O' '.' 'O' '.' '.' '.' '.' '#' '.' '.' '.' '.' 'O' '.' '.'
  'O' 'O' '.' '#' 'O' '#' '.' '.' 'O' '.' '.' '.' '.' '#']
 ['#' '.' '.' '.' '.' '.' '.' '.' '.' '.' '.' 'O' 'O' '.' '.' '.' '.' 'O'
  '.' '.' '#' 'O' 'O' '.' '.' '.' 'O' '.' 'O' '.' 'O' '.' '.' '.' '.' '#'
  '.' '.' 'O' '.' '.' '.

## PART 1

In [ ]:
movement_map = {'^': (-1, 0), 'v': (1, 0), '<': (0, -1), '>': (0, 1)}

def apply_movement(position, movement):
    return tuple(np.array(position) + np.array(movement))

def move_robot(warehouse_map, robot_position, robot_instruction):
    movement = movement_map[robot_instruction]
    next_robot_position = apply_movement(robot_position, movement)
    print(next_robot_position)
    if warehouse_map[next_robot_position] == '.':
        return warehouse_map, next_robot_position
    if warehouse_map[next_robot_position] == '#':
        return warehouse_map, robot_position
    if warehouse_map[next_robot_position] == 'O':
        next_package_position = apply_movement(next_robot_position, movement)
        # Jump over entire package line
        while warehouse_map[next_package_position] == 'O':
            next_package_position = apply_movement(next_package_position, movement)
        # What is the first location after the package line?
        # If it is a block the robot does not move
        if warehouse_map[next_package_position] == '#':
            return warehouse_map, robot_position
        # If there's empty space move entire package block 1 position: update new end of line and new beginning of line
        if warehouse_map[next_package_position] == '.':
            warehouse_map[next_package_position] = 'O'
            warehouse_map[next_robot_position] = '.'
            return warehouse_map, next_robot_position
        raise ValueError(f"Unknown location at positon {next_package_position}: {warehouse_map[next_package_position]}")
    return warehouse_map, robot_position

def move_robot_alllist(warehouse_map, robot_position, robot_instructions_list):
    robot_moved_position = robot_position
    for robot_instruction in robot_instructions_list:
        warehouse_map, robot_moved_position = move_robot(warehouse_map, robot_moved_position, robot_instruction)
    return warehouse_map, robot_moved_position

In [22]:
# Solve part 1
final_warehouse_map, final_robot_position = move_robot_alllist(warehouse_map, robot_position, robot_instructions_list)
boxes_positions = np.array(list(zip(np.where(final_warehouse_map == "O")[0], np.where(final_warehouse_map == "O")[1])))
boxes_gps = np.sum(np.dot(boxes_positions, np.array([100, 1])))
print(boxes_positions,boxes_gps,100*final_robot_position[0] + final_robot_position[1])

(np.int64(23), np.int64(24))
(np.int64(22), np.int64(24))
(np.int64(23), np.int64(24))
(np.int64(24), np.int64(24))
(np.int64(24), np.int64(25))
(np.int64(24), np.int64(23))
(np.int64(24), np.int64(23))
(np.int64(24), np.int64(25))
(np.int64(24), np.int64(23))
(np.int64(24), np.int64(25))
(np.int64(24), np.int64(23))
(np.int64(23), np.int64(24))
(np.int64(24), np.int64(24))
(np.int64(23), np.int64(24))
(np.int64(24), np.int64(24))
(np.int64(23), np.int64(24))
(np.int64(24), np.int64(24))
(np.int64(24), np.int64(23))
(np.int64(23), np.int64(24))
(np.int64(24), np.int64(24))
(np.int64(25), np.int64(24))
(np.int64(26), np.int64(24))
(np.int64(25), np.int64(25))
(np.int64(25), np.int64(24))
(np.int64(26), np.int64(24))
(np.int64(25), np.int64(25))
(np.int64(26), np.int64(25))
(np.int64(26), np.int64(26))
(np.int64(27), np.int64(26))
(np.int64(27), np.int64(27))
(np.int64(26), np.int64(27))
(np.int64(27), np.int64(27))
(np.int64(27), np.int64(26))
(np.int64(27), np.int64(25))
(np.int64(28),

In [46]:
warehouse_map[tuple(robot_position)]
next_robot_position = tuple(np.array(robot_position) + np.array(movement_map['>']))
print(next_robot_position)
warehouse_map[next_robot_position]

(np.int64(2), np.int64(3))


np.str_('.')

## PART 2

In [132]:
def print_warehouse_map(warehouse_map):
    for line in warehouse_map:
        print(''.join(line))

In [141]:
scale_up_mapping = {'#': '##', 'O': '[]', '.': '..', '@': '@.'}
warehouse_map_with_robot = warehouse_map.copy()
warehouse_map_with_robot[robot_position] = '@'
scaled_up_warehouse_map = np.array([[y for y in ''.join([scale_up_mapping[x] for x in line])] for line in warehouse_map_with_robot])

robot_position_scaled = tuple(np.array(np.where(scaled_up_warehouse_map == "@")).flatten())
print_warehouse_map(scaled_up_warehouse_map)
scaled_up_warehouse_map[robot_position_scaled] = '.'
print_warehouse_map(scaled_up_warehouse_map)
print(robot_position_scaled)

####################################################################################################
##......####......[][][]..##..[]..[][][]............[]######......[]..##....[]..............[]....##
##........[]......[]..........[]....[]##[]##[]..[]........[]......[][]##[][][][]........[][][]....##
##....[]..[]....[]..##....##......[]......[]..[]........##........[]....[][]..##[]##....[]........##
##....................[][]........[]....##[][]......[]..[]..[]........##....[]..............##..####
##[][]..............[]..........[][]..........[]..........[]..[]..##......[]....[]....[]##..##....##
##........##....[]..##..........[][][]..[]............[]........[][]..[]##[][]..[]......[]..##..[]##
##..[]##..[][]........[]##......[][]......[]..[]....................[]..................[][]....[]##
##[][]..[]##..[]......[]..[]....[]..........[]##[]..[]....[]......[][][]......[]........##[][][][]##
##....[][]..[]................[]....##........[][][][][]....[][]......[]##....[][]####.....

In [134]:
movement_type = {'<': 'horizontal', '>': 'horizontal', '^': 'vertical', 'v': 'vertical'}

column_shift = {'[': 1, ']': -1}

def move_robot_widemap(warehouse_map, robot_position, robot_instruction):
    movement = movement_map[robot_instruction]
    next_robot_position = apply_movement(robot_position, movement)
    next_warehouse_map = warehouse_map.copy()
    #print(next_robot_position)
    position_content = warehouse_map[next_robot_position]
    if position_content == '.':
        return warehouse_map, next_robot_position
    if position_content == '#':
        return warehouse_map, robot_position
    
    # What if there's a package?
    if position_content in ['[',']']:

        # HORIZONTAL MOVEMENT
        if movement_type[robot_instruction] == 'horizontal':
            next_package_position = apply_movement(next_robot_position, movement)
            next_position_content = warehouse_map[next_package_position]
            while next_position_content in ['[', ']']:
                next_warehouse_map[next_package_position] = position_content
                position_content = next_position_content
                next_package_position = apply_movement(next_package_position, movement)
                next_position_content = warehouse_map[next_package_position]

            # What is the first location after the package line?
            # If it is a block the robot does not move -- return original map
            if next_position_content == '#':
                return warehouse_map, robot_position
            # If there's empty space move entire package block 1 position: update new map and return it
            if next_position_content == '.':
                next_warehouse_map[next_package_position] = position_content
                next_warehouse_map[next_robot_position] = '.'
                return next_warehouse_map, next_robot_position
            raise ValueError(f"Unknown location at positon {next_package_position}: {warehouse_map[next_package_position]}")
        
        # VERTICAL MOVEMENT
        if movement_type[robot_instruction] == 'vertical':
            
            # Get all positions of the pushed/pushing box (next row and next multiple columns)
            pushing_box_columns = set([next_robot_position[1]])
            pushing_box_columns.add(next_robot_position[1] + column_shift[position_content])
            pushing_box_row = next_robot_position[0]

            # Vacate pushing boxes locations
            for column in pushing_box_columns:
                next_warehouse_map[pushing_box_row, column] = '.'

            while len(pushing_box_columns) > 0:
                this_step_pushing_box_columns = pushing_box_columns.copy()
                # Check which boxes will be pushed in turn: pushed_box_row and pushing_box_columns
                pushed_box_row = pushing_box_row + movement[0]
                pushed_box_columns = set()
                for column in this_step_pushing_box_columns:
                    pushed_content = warehouse_map[pushed_box_row, column]

                    # Obstacle encountered, nothing changes
                    if pushed_content == '#':
                        return warehouse_map, robot_position

                    # Another box pushed, propagete the pushing
                    if pushed_content in ['[',']']:
                        # Get the 2 columns occupied by the box as 'pushed'
                        pushed_box_columns.add(column)
                        pushed_box_columns.add(column + column_shift[pushed_content])

                # Vacate pushed boxes locations
                for column in pushed_box_columns:
                    next_warehouse_map[pushed_box_row, column] = '.'
                
                # Advance pushing boxes into pushed row
                for column in this_step_pushing_box_columns:
                    next_warehouse_map[pushed_box_row, column] = warehouse_map[pushing_box_row, column]
                
                # The pushed boxes now become the pushing boxes!
                pushing_box_columns = pushed_box_columns
                pushing_box_row = pushed_box_row
            # If we didn't return yet that means we did not meet an obstacle, so we return the moved map
            return next_warehouse_map, next_robot_position
        
def move_robot_widemap_alllist(warehouse_map, robot_position, robot_instructions_list):
    robot_moved_position = robot_position
    for robot_instruction in robot_instructions_list:
        warehouse_map, robot_moved_position = move_robot_widemap(warehouse_map, robot_moved_position, robot_instruction)
    return warehouse_map, robot_moved_position

In [142]:
# Solve part 2
final_warehouse_map, final_robot_position = move_robot_widemap_alllist(scaled_up_warehouse_map, robot_position_scaled, robot_instructions_list)
# Compute GPS
boxes_positions = np.array(list(zip(np.where(final_warehouse_map == "[")[0], np.where(final_warehouse_map == "[")[1])))
boxes_gps = np.sum(np.dot(boxes_positions, np.array([100, 1])))
print(boxes_gps)

1489116


In [140]:
print_warehouse_map(final_warehouse_map)

####################
##[][][][][][][][]##
##............[][]##
##..............[]##
##[]..............##
##[]##............##
##[][]............##
##[]..............##
##[]......[][][][]##
####################
